In [1]:
# Shell Plus Model Imports
from chirps.models import Chirp, Tag
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Count, F, Max, Min, Sum, Q, Prefetch, Case, When
from django.core.urlresolvers import reverse
from django.utils import timezone
from django.conf import settings
from django.core.cache import cache

In [2]:
Chirp.objects.all()

[<Chirp: jeff says  Boring>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  I work at Wendy's>]

In [3]:
chirps = Chirp.objects.all()

In [4]:
type(chirps)

django.db.models.query.QuerySet

In [5]:
[chirp.subject for chirp in chirps]

[' Boring', ' I am smart', " I work at Wendy's"]

# Creating an model object

In [6]:
chirp = Chirp()

In [8]:
chirp.subject = "I am a simple chirp"
chirp.message = "I am a simple chirp"

In [9]:
user = User.objects.get(pk=1)

In [10]:
user

<User: jeff>

In [11]:
chirp.user = user
chirp.save()

In [12]:
chirp.id

7

In [13]:
Chirp.objects.create(user=user, subject="Create method chirp",
                    message="I have created my first chirp from create")

<Chirp: jeff says Create method chirp>

# Filtering

In [15]:
Chirp.objects.get(pk=7)

<Chirp: jeff says I am a simple chirp>

In [17]:
Chirp.objects.filter(pk=7)[0]

django.db.models.query.QuerySet

In [19]:
Chirp.objects.get(subject="I am a simple chirp")

<Chirp: jeff says I am a simple chirp>

In [24]:
Chirp.objects.create(message="I am having fun", subject="I am having fun",
                    user=user)

<Chirp: jeff says I am having fun>

In [25]:
Chirp.objects.get(subject__startswith="I")

MultipleObjectsReturned: get() returned more than one Chirp -- it returned 2!

In [26]:
Chirp.objects.filter(subject__startswith="I")

[<Chirp: jeff says I am a simple chirp>, <Chirp: jeff says I am having fun>]

In [29]:
Chirp.objects.filter(subject__exact="I am a simple chirp")

[<Chirp: jeff says I am a simple chirp>]

In [30]:
Chirp.objects.filter(subject__iexact="i am a simple chirp")

[<Chirp: jeff says I am a simple chirp>]

In [31]:
Chirp.objects.filter(subject__contains="simple")

[<Chirp: jeff says I am a simple chirp>]

In [32]:
Chirp.objects.filter(subject__icontains="SIMPLE")

[<Chirp: jeff says I am a simple chirp>]

In [33]:
Chirp.objects.filter(subject__endswith="p")

[<Chirp: jeff says I am a simple chirp>, <Chirp: jeff says Create method chirp>]

# Spanning relationships

In [34]:
chirp = Chirp.objects.first()

In [35]:
chirp.user

<User: jeff>

In [36]:
chirp.user.username

'jeff'

In [37]:
jeff = User.objects.first()

In [38]:
dir(jeff)

['DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 'REQUIRED_FIELDS',
 'USERNAME_FIELD',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_manager',
 '_check_column_name_clashes',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_ordering',
 '_check_swappable',
 '_check_unique_together',
 '_default_manager',
 '_deferred',
 '_do_insert',
 '_do_update',
 '_get_FIELD_display',
 '_get_next_or_previous_by_FIELD',
 '_get_next_or_previous_in_order',
 '_get_pk_val',
 '_get_unique_checks',
 '_

In [39]:
type(jeff.chirp_set)

django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager

In [40]:
jeff.chirp_set.all()

[<Chirp: jeff says  Boring>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says I am a simple chirp>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am having fun>]

In [41]:
User.objects.create_user(username="kevin", password="password",
                        email="kevin@kevin.org")

<User: kevin>

In [43]:
kevin = User.objects.get(username="kevin")

In [44]:
kevin.chirp_set.all()

[]

In [45]:
Chirp.objects.create(subject="YOLO", message="I ate too many chicken nuggets",
                    user=kevin)

<Chirp: kevin says YOLO>

In [47]:
kevin.chirp_set.create(subject="Goal", message="Go to space")

<Chirp: kevin says Goal>

In [48]:
Chirp.objects.filter(user__email__iexact="kevin@kevin.org")

[<Chirp: kevin says YOLO>, <Chirp: kevin says Goal>]

# Limiting Results

In [49]:
Chirp.objects.all()[:2]

[<Chirp: jeff says  Boring>, <Chirp: jeff says  I am smart>]

In [50]:
Chirp.objects.all()[2:4]

[<Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says I am a simple chirp>]

In [51]:
Chirp.objects.all()[::-1]

[<Chirp: kevin says Goal>,
 <Chirp: kevin says YOLO>,
 <Chirp: jeff says I am having fun>,
 <Chirp: jeff says Create method chirp>,
 <Chirp: jeff says I am a simple chirp>,
 <Chirp: jeff says  I work at Wendy's>,
 <Chirp: jeff says  I am smart>,
 <Chirp: jeff says  Boring>]

In [52]:
Chirp.objects.all()[::2]

[<Chirp: jeff says  Boring>,
 <Chirp: jeff says  I work at Wendy's>,
 <Chirp: jeff says Create method chirp>,
 <Chirp: kevin says YOLO>]

In [53]:
import random

In [56]:
random.choice(Chirp.objects.all())

<Chirp: jeff says  I work at Wendy's>

# Order by

In [57]:
Chirp.objects.all().order_by("created_at")

[<Chirp: jeff says  Boring>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says I am a simple chirp>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am having fun>, <Chirp: kevin says YOLO>, <Chirp: kevin says Goal>]

In [58]:
Chirp.objects.all().order_by("-created_at")

[<Chirp: kevin says Goal>, <Chirp: kevin says YOLO>, <Chirp: jeff says I am having fun>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am a simple chirp>, <Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  Boring>]

In [59]:
Chirp.objects.all().order_by("-created_at", "message")

[<Chirp: kevin says Goal>, <Chirp: kevin says YOLO>, <Chirp: jeff says I am having fun>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am a simple chirp>, <Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  Boring>]

In [61]:
Chirp.objects.all().order_by("-created_at")[0:5]

[<Chirp: kevin says Goal>, <Chirp: kevin says YOLO>, <Chirp: jeff says I am having fun>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am a simple chirp>]

# F() ing objects

In [63]:
Chirp.objects.create(user=user, subject="YOLO", message="@kevin is yoloing")

<Chirp: jeff says YOLO>

In [64]:
Chirp.objects.create(user=user, subject="YOLO", message="jeff is playin it safe")

<Chirp: jeff says YOLO>

In [65]:
Chirp.objects.filter(message_icontains=user__username)

NameError: name 'user__username' is not defined

In [66]:
Chirp.objects.filter(message__icontains=F("user__username"))

[<Chirp: jeff says YOLO>]

# Q() ing Objects

In [69]:
Chirp.objects.filter(subject__startswith="I").filter(subject__endswith="n")

[<Chirp: jeff says I am having fun>]

In [71]:
Chirp.objects.filter(Q(message__startswith="jeff") | Q(message__startswith="@"))

[<Chirp: jeff says YOLO>, <Chirp: jeff says YOLO>]

# Querysets are unique

In [73]:
q1 = Chirp.objects.filter(user__username='jeff')
q2 = q1.exclude(subject__icontains='YOLO')

# Querysets are Lazy

In [74]:
q = Chirp.objects.filter(user__username='jeff')
q = q.exclude(subject__icontains='YOLO')

In [75]:
q

[<Chirp: jeff says  Boring>, <Chirp: jeff says  I am smart>, <Chirp: jeff says  I work at Wendy's>, <Chirp: jeff says I am a simple chirp>, <Chirp: jeff says Create method chirp>, <Chirp: jeff says I am having fun>]

# Aggregates

In [76]:
user

<User: jeff>

In [77]:
user.chirp_set.count()

8

In [90]:
user.chirp_set.aggregate(chirp_count=Count('id'))

{'chirp_count': 8}

In [94]:
users = User.objects.annotate(chirp_count=Count("chirp__id"))

In [98]:
users[1].chirp_count

2

# Many to Many

In [2]:
chirp = Chirp.objects.first()

In [3]:
chirp

<Chirp: jeff says  Boring>

In [5]:
chirp.tag_set.all()

[]

In [7]:
chirp.tag_set.create(name="boring")

<Tag: Name: boring Posted At: 2016-03-24 18:35:31.226299+00:00>

In [9]:
chirp.tag_set.all()

[<Tag: Name: boring Posted At: 2016-03-24 18:35:31.226299+00:00>]

In [10]:
Tag.objects.create(name="Happy")

<Tag: Name: Happy Posted At: 2016-03-24 18:36:30.926992+00:00>

In [13]:
tag = Tag.objects.get(name="Happy")

In [14]:
chirp.tag_set.add(tag)